In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
import torchaudio as ta
import torchaudio.functional as taf
import torchaudio.transforms as tat
from torchvision import transforms

print(torch.__version__)        
print(ta.__version__)

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Audio, display

import pandas as pd
import os
import pprint

from typing import *
import itertools
from collections import Counter

import numpy as np
from datetime import datetime
from lark.config import Config
from lark.learner import Learner
from lark.ops import Sig2Spec

1.8.1
0.8.0a0+e4e171a


In [3]:
import torch
# get list of models
torch.hub.list('zhanghang1989/ResNeSt', force_reload=True)

Downloading: "https://github.com/zhanghang1989/ResNeSt/archive/master.zip" to /home/koen/.cache/torch/hub/master.zip


['resnest101',
 'resnest200',
 'resnest269',
 'resnest50',
 'resnest50_fast_1s1x64d',
 'resnest50_fast_1s2x40d',
 'resnest50_fast_1s4x24d',
 'resnest50_fast_2s1x64d',
 'resnest50_fast_2s2x40d',
 'resnest50_fast_4s1x64d',
 'resnest50_fast_4s2x40d']

In [177]:
cfg = Config(
#     n_mels = 128,
#     f_min = 20,
#     f_max = 16000,
#     n_fft = 2048,
#     hop_length = 512,
    
    noise_nsr_dbs = [40, 30, 20, 10, 3],

    site='SSW', 
    use_neptune=False, 
    n_epochs=10, 
    bs=32, 
    lr=1e-3, 
    model='resnest50',
)

In [178]:
cfg.as_dict()

{'site': 'SSW',
 'data_dir': 'data/birdclef-2021',
 'checkpoint_dir': 'checkpoints',
 'bs': 32,
 'n_workers': 12,
 'training_dataset_size': 10752,
 'duration': 5,
 'use_noise': True,
 'noise_nsr_dbs': [40, 30, 20, 10, 3],
 'noise_dir': 'data/noise/BirdVox-DCASE-20k/wav-32k',
 'use_overlays': True,
 'max_overlays': 5,
 'overlay_weights': [0.71986223,
  0.21010333,
  0.06314581,
  0.00574053,
  0.00114811],
 'overlay_snr_dbs': [20, 10, 3],
 'use_neptune': False,
 'sr': 32000,
 'n_frames': 160000,
 'n_fft': 512,
 'window_length': 512,
 'n_mels': 64,
 'hop_length': 312,
 'f_min': 150,
 'f_max': 15000,
 'lr': 0.001,
 'n_epochs': 10,
 'model': 'resnest50',
 'optimizer': 'torch.optim.Adam',
 'loss_fn': 'torch.nn.BCEWithLogitsLoss',
 'scheduler': 'torch.optim.lr_scheduler.CosineAnnealingLR',
 'labels': ['amecro',
  'amegfi',
  'amerob',
  'balori',
  'belkin1',
  'bkcchi',
  'blujay',
  'cangoo',
  'chswar',
  'comgra',
  'comyel',
  'dowwoo',
  'eastow',
  'eawpew',
  'gockin',
  'grycat',
  

In [184]:
prep = torch.nn.Sequential(
    Sig2Spec(cfg, forward_as_image=True),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)

main_model = rn_model = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)
del main_model.fc
for param in main_model.parameters():
    param.requires_grad = True

posp = torch.nn.Sequential(
    torch.nn.Linear(in_features=2048, out_features=1024, bias=True),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features=1024, out_features=len(cfg.labels), bias=True)
)
for param in posp.parameters():
    param.requires_grad = True


main_model.fc = posp

model = torch.nn.Sequential(prep, main_model)
model = model.cuda()
# model

# model = torch.nn.Sequential(Sig2Spec(cfg, forward_as_image=True), preprocess, rn_model)
# model[1].fc = torch.nn.Linear(in_features=2048, out_features=len(cfg.labels), bias=True)

Using cache found in /home/koen/.cache/torch/hub/zhanghang1989_ResNeSt_master


In [185]:
lrn = Learner("resnest-test", cfg, model)

In [186]:
# lrn.loss_fn = torch.nn.BCEWithLogitsLoss().cuda()
# lrn.loss_fn = torch.nn.BCELoss().cuda()

In [187]:
# lrn.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(lrn.optimizer, eta_min=1e-5, T_max=cfg.n_epochs)

In [188]:
lrn.learn()

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

epoch:   1 train loss: 0.199318 train f1: 0.000257 valid loss: 0.130179 valid f1: 0.210526


  0%|          | 0/336 [00:00<?, ?it/s]

...

  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

epoch:   9 train loss: 0.186054 train f1: 0.022559 valid loss: 0.144759 valid f1: 0.082876


  0%|          | 0/336 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [157]:
for t in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    print(t, lrn.valid_loop(t)[1])

  0%|          | 0/38 [00:00<?, ?it/s]

0.0 0.04363638157711217


  0%|          | 0/38 [00:00<?, ?it/s]

0.1 0.16626332830147525


  0%|          | 0/38 [00:00<?, ?it/s]

...

0.8 0.2634182758629322


  0%|          | 0/38 [00:00<?, ?it/s]

0.9 0.31236892664118815


  0%|          | 0/38 [00:00<?, ?it/s]

1.0 0.21052631578947367


In [158]:
lrn.load_checkpoint()

{'epoch': 40,
 'valid_loss': 0.08395799808204174,
 'valid_score': 0.5491774180217793}

In [159]:
for t in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    print(t, lrn.valid_loop(t)[1])

  0%|          | 0/38 [00:00<?, ?it/s]

0.0 0.04363638157711217


  0%|          | 0/38 [00:00<?, ?it/s]

0.1 0.2623506858944893


  0%|          | 0/38 [00:00<?, ?it/s]

...

0.8 0.43307367918130596


  0%|          | 0/38 [00:00<?, ?it/s]

0.9 0.38294041754775926


  0%|          | 0/38 [00:00<?, ?it/s]

1.0 0.21052631578947367
